# Analysis Plan
* Import the 3 files into databricks.
* In the following sections, i will use both Sql and Pandas to work on the problem.
* Since databricks creates pyspark dataframe, i transform it into pandas for easy manipulation.
* After initial reading on the data, i would focus on a few things for each table:
  - transaction:
    - missing values in each column 
    - purchase by date, scan by date , by store
    - distinct stores, users, scans 
    - duplicate scan of receipt
    - relation between barcode, final quantity and final sale columns

  - User:
    - tenure for customer 
    - any dob issues, 
    - gender distribution, state distribution, language distribution, 
    - any missing values

  - Product:
    - Identify the hireachy in the 4 categories
    - missing of manufacture and brand 
    - barcard associate with each brand

* after initial understanding of the tables, need to join the transaction with user to get user_join_date and then use barcode to join with product to understand product category. 
  - a few analysis we can put together
    - preferred category for each user
    - preferred store for each user
    - avg freq/ dollar value generated from each user
    - any scan time preference 
    - top manufacture's scan frequency, dollar value, store distribution
    - invalid barcode? or invalid scan



In [0]:
import pandas as pd
import numpy as np

In [0]:
transaction_df = spark.sql("SELECT * FROM transaction").toPandas()

In [0]:
transaction_df.head(5)

# transaction: missing values


In [0]:
print(transaction_df.shape)
transaction_df.describe()
# so barcode and final sale both have some missing values.
# the final quantity column should be a number, so we need to check that

In [0]:
%sql
select distinct(FINAL_QUANTITY), count(*) as cnt from transaction
group by 1
order by cnt
-- the final quantity field should be an integer, however the data has outlier values such as 1.24, 1.44 etc, this seems to be some data issues
-- majority of the quantity is 1 and 0, let's transform zero to 0 and transform this field to a decimal field

In [0]:
transaction_df.FINAL_QUANTITY.replace('zero', 0, inplace=True)
transaction_df.FINAL_QUANTITY = pd.to_numeric(transaction_df.FINAL_QUANTITY)

In [0]:
transaction_df.head(5)

In [0]:
# Missing in each Column
nan_counts = transaction_df.isna().sum()
nan_counts
## so there are no missing values in most fields, other than barcode (5762) and final sale (12500)

# transaction: duplicate record

In [0]:
#distinct stores, users, BARCODES, receipts
print(transaction_df.RECEIPT_ID.nunique()) ##24440
print(transaction_df.USER_ID.nunique()) ##17694
print(transaction_df.STORE_NAME.nunique()) ##954
print(transaction_df.BARCODE.nunique()) ##11027

In [0]:
%sql
select count(*) from (
  select distinct * from transaction
)
-- there are 171 records that are pure duplicates 

In [0]:
%sql
-- what does duplicate receipt mean
select receipt_id, count(receipt_id) from transaction group by 1 having count(RECEIPT_ID) > 1 order by count(RECEIPT_ID) desc

In [0]:
%sql
select distinct * from transaction where RECEIPT_ID = 'bedac253-2256-461b-96af-267748e6cecf'
-- this receipt record has 12 rows, 3 distinct rows. 

In [0]:
%sql
-- check one assumption: every receipt only have one row that's valid, which has a non 'zero' quantity and not null sale amount
select count(distinct receipt_id )from transaction 
where FINAL_QUANTITY != 'zero' and FINAL_SALE is not null
--24440
-- that is the same amount as the distinct receipt in the transaction table, the assumption is correct

## dedup on the receiptID

In [0]:
%sql
create or replace table new_transaction as
select * from transaction where FINAL_QUANTITY != 'zero' and FINAL_SALE is not null

In [0]:
new_transaction_df = spark.sql("SELECT * FROM new_transaction").toPandas()

In [0]:
new_transaction_df.describe()

In [0]:
new_transaction_df.isna().sum()
# barcode 2881

# transaction: count of receipt by dimensions

In [0]:
new_transaction_df.head(5)

In [0]:
%sql
select * from new_transaction
where BARCODE <= 0
-- 4 records

In [0]:
%sql
--is there any receipt belongs to two different user
select receipt_id, count(distinct USER_ID) from new_transaction
group by 1
having count(distinct USER_ID) > 1
-- no

In [0]:
%sql
select STORE_NAME	, count(RECEIPT_ID) from new_transaction
where BARCODE is null
group by 1
order by  count(RECEIPT_ID)
-- there is no specific day that have a lot of null barcode
-- however, ~40% null barcode are coming from ALDI and CVS store

In [0]:
%sql
select PURCHASE_DATE, count(*) from new_transaction group by 1 order by 1

In [0]:
new_transaction_df.groupby('PURCHASE_DATE')['RECEIPT_ID'].count().plot(kind='line')
## Looks like there is a decrese in purchasing 

In [0]:
%sql
select date(SCAN_DATE), count(*) from new_transaction group by 1 order by 1

In [0]:
from datetime import datetime
new_transaction_df['SCAN_DATE_YMD'] = new_transaction_df['SCAN_DATE'].dt.strftime("%Y-%m-%d")
# new_transaction_df.head(10)
new_transaction_df.groupby('SCAN_DATE_YMD')['RECEIPT_ID'].count().plot(kind='line')
## Other than the first day and outlier day 2024-07-01, the scan and purchase seems to be consistent with each other

## Scan date 2024-07-01 has a spike, what happened

In [0]:
%sql
select date(SCAN_DATE) as scan_ymd, count(distinct STORE_NAME) from new_transaction
group by 1

In [0]:
%sql
select count(distinct STORE_NAME) from new_transaction
where date(SCAN_DATE) = '2024-07-01'
-- 550 scans from 536 users, but from 91 stores which is the highest in the three month data 

In [0]:
%sql
select date(SCAN_DATE) , STORE_NAME, count(*) from new_transaction
-- where date(SCAN_DATE) = '2024-07-01'
group by 1,2
order by 1, count(*) desc
-- on '2024-07-01', almost 150 more scans from Walmart and ALDI

In [0]:
%sql
-- what's the date difference between purchase and scan
select diff, count(receipt_id) from (
  select RECEIPT_ID, date_diff(SCAN_DATE, PURCHASE_DATE) AS diff 
  from new_transaction
)
group by 1
order by 1
--  47 receipts have scan before purchase

In [0]:
%sql
select  STORE_NAME, count(*) from new_transaction
group by 1
order by  count(*) desc
limit 10
-- walmart is the biggest store partner, followed by DOLLAR GENERAL STORE

In [0]:
%sql
select  USER_ID, count(*) from new_transaction
group by 1
order by  count(*) desc
limit 10
-- User distribution seems reasonable, no significant outlier 

# Data Issues identified in Transaction table
- The final quantity field contains non integer value, and zero needs to be transformed to 0 to ensure this field's data quality
- The missing value in barcode is about 11% and in final sale is about 25%
- finding: There are duplicate rows and duplicate receipt ID with similar information in the table. after investigate the table, each receipt ID have multiple rows where only one of them should be the actual receipt w/ correct quantity and correct sale amount. 

- after dedup, the barcode column still have ~12% missing values and invalid input (such as -1). The missing barcode happens across all days but mostly show up in ALDI and CVS stores. 
- there are 47 records have a scan date before purchase date, which should be invalid too.



# User: missing values

In [0]:
user_df = spark.sql("SELECT * FROM user").toPandas()

In [0]:
user_df.head(5)

In [0]:
user_df.shape
#(100000, 6)

In [0]:
user_df.isna().sum()

# User: distribution

In [0]:
%sql
-- No duplicate ID 
create or replace view user_tenure_age as 
select id,  STATE, LANGUAGE, GENDER, tenure_month, signup_age,created_day, birthday,
case when tenure_month < 0 then '0 - Special'
     when tenure_month >= 0 and tenure_month <= 12 then '1 - 0-12M'
     when tenure_month > 12 and tenure_month <= 24 then '2 - 12-24M'
     else '3 - 24M+'
     end as Tenure_grp,
case when signup_age <= 0 then '0 - Special'
     when signup_age > 0 and signup_age < 10 then '1 - Under 10'
     when signup_age >= 10 and signup_age < 18 then '2 - 10-17'
     when signup_age >= 18 and signup_age < 30 then '3 - 18-29'
     when signup_age >= 30 and signup_age < 45 then '4 - 30-44'
     when signup_age >= 45 and signup_age < 100 then '5 - 45-99'
     else '6 - 100+'
     end as Age_grp
from (   
  select id, STATE, LANGUAGE, GENDER,  date(created_date) as created_day, date(BIRTH_DATE) as birthday,
  round(date_diff('2024-09-08', date(created_date))/30,1) as tenure_month,
  round(date_diff(date(CREATED_DATE),date(BIRTH_DATE)) / 365,1) as signup_age
  from user
)

In [0]:
user_grp_df = spark.sql("SELECT * FROM user_tenure_age").toPandas()

In [0]:
user_grp_df.groupby('Tenure_grp')['id'].count().plot(kind='bar')

In [0]:
user_grp_df.groupby('Age_grp')['id'].count().plot(kind='bar')

In [0]:
%sql
select tenure_grp, Age_grp, count(*) from user_tenure_age
group by 1,2
order by 1,2

In [0]:
%sql
select * from user_tenure_age
where Tenure_grp = '0 - Special'
-- this is fine since the Sep 08 is just the max date in the transaction. 

In [0]:
%sql
select gender, count(*) from user_tenure_age
group by 1
-- gender has duplicate values, need better definition

In [0]:
%sql
select state, count(*) from user_tenure_age
group by 1
order by count(*) desc
-- state looks good

In [0]:
%sql
select language, count(*) from user_tenure_age
group by 1
order by count(*) desc
-- language has a lot missing, is this a mandantory field? 

# Data Issues identified in User table
- All columns except ID and created date has some missing, language missing the most
- why would the birth date have HHMMSS?
- There are 57 records that have a birth date after the create date or under 10 year old at creation. They are all tenured customers.
- in every tenure group, there are customers who claimed to be 100 years old +, probably need some attention.
- the gender column have duplicated values, such as prefer_not_to_say and Prefer not to say, My gender isn't listed and not_listed. Need clarify on this field.



# Product: missing values

In [0]:
product_df = spark.sql("SELECT * FROM products").toPandas()

In [0]:
product_df.head(5)

In [0]:
product_df.shape
#(845552, 7)

In [0]:
product_df.isna().sum()

# Product: Hireachy on category

In [0]:
%sql
select * from products
where CATEGORY_1 is null
-- if category 1 is missing, the rest are all missing. which might indicate category 1 is the highest hireachy of product category, but still hard to understand
-- and looking at the examples here, these brands are popular ones, where i checked on the brand and other barcodes would have the category. I think at least the category 1 should be matched to the brand. 

In [0]:
%sql
select  category_1, CATEGORY_2, CATEGORY_3, CATEGORY_4, count(*) from products
group by 1,2,3,4
order by 1,2,3,4

# Product: missing barcode

In [0]:
%sql
select * from products
where MANUFACTURER is null

In [0]:
%sql
select * from products
where barcode is null

In [0]:
%sql
select brand, count(*), count(case when barcode is null then 1 end) as barcode_missing_cnt, 
round(count(case when barcode is null then 1 end) / count(*),2) * 100 as missing_barcode_pct
 from products
-- where barcode is null
group by 1
order by missing_barcode_pct desc

# Product: relation between manufacture, brand and category

In [0]:
%sql
select CATEGORY_1, MANUFACTURER, brand, count(*) from products
group by 1,2,3
order by count(*) desc

In [0]:
%sql
-- could there be a brand that belong to multiple manufacture

select brand, count(distinct manufacturer) from products
group by 1
having count(distinct manufacturer) > 1

In [0]:
%sql
select distinct MANUFACTURER, brand from products
where brand in ('LE PETIT MARSEILIAIS','TYGAZ','CHAPSTICK')


In [0]:
%sql
-- could there be a brand that belong to multiple category_1

select brand, count(distinct category_1) from products
group by 1
having count(distinct category_1) > 1

In [0]:
%sql
-- could there be a MANUFACTURER that belong to multiple category_1

select MANUFACTURER, count(distinct category_1) from products
group by 1

having count(distinct category_1) > 1
order by count(distinct category_1) desc

In [0]:
%sql
select distinct category_1, manufacturer from products
where MANUFACTURER in ('KEURIG DR PEPPER','THE COCA-COLA COMPANY','PEPSICO')

# Data Issues identified in Product table
- All columns have missing values, manufacture and brand are 1-1 match on missing(27%).
- if category 1 is missing, the rest categories are all missing. which might indicate category 1 is the highest hireachy of product category, but still hard to understand. The hireachy on product need better definition. 
- Category 1 have a 'Needs Review' value, super confusing
- there are 69 brands that have no barcode for all products they have, which doesn't make sense. And another 100 have like half of products with no barcode
- there are brand value = "brand not known"  and manufacture = 'Unknown'
- there are three brands that belongs to multiple manufacture
- there are many brand/ manufacture falls into 2 category_1, the definition of category_1 is very unclear
- following on the previous point, the most multiple category per manufacture are some beverage company like keurig dr pepper, pepsi, and their category_1 is definitely wild




# Summarization on Part 1

**Are there any data quality issues present?**

-  There are missing values in many columns, the most need attention one is the barcode missing in both transaction and product table
- The transaction table has many duplicate records, after the investigation, my assumption is when the receipt ID have both quantity (not zero) and sale amount present, that's the row we should keep for this receipt in the transaction table.
- The final quantity field in the transaction table contain non-interger value which i am not sure if that's correct.
- There are transaction records that have a scan date before purchase date.
- There are user records that have a birth date after the create account date, or under 10 year old or above 100 year old at creation.
- The gender column have similar but distinct values, which could be combined to make it clear.
- The product category columns are very confusing with so many missing values. 
- The missing barcode is concerning in the product table, there are 69 brands that have no barcode for all products they have.
- Category contains value like "Need Review". Brand value is "brand not known" and manufacture is 'Unknown'.
- There are three brands that belongs to multiple manufacture
- There are manufactures that belongs to multiple categories 


Are there any fields that are challenging to understand?

- Final quantity in transaction table : non interger value exist
- Purchase date and scan date: Assumption is Purchase date means the date on the receipt, scan date is the date the receipt is uploaded to fetch
- DOB in user table: why it has HHMMDD
- Product Category 1,2,3,4 in Product table: What's the relation between them, is there a hireachy between them, what does each category represent.
- Product Barcode in Product: What does Barcode represent, does it mean item number